In [1]:
import gym
import fh_ac_ai_gym

In [2]:
def get_neighbors(x, y):
    neighbors = []
    for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < 4 and 0 <= ny < 4:
            neighbors.append((nx, ny))
    return neighbors

In [3]:
class KnowledgeBase_Horn:
    def __init__(self):
        self.facts = set()
        self.rules = []
    def tell(self, sentence):
        sentence = sentence.strip()
        if "=>" in sentence:
            premise_str, conclusion = sentence.split("=>")
            premises = set(p.strip() for p in premise_str.split("&") if p.strip())
            negation = f"-{conclusion}" if conclusion[0] != "-" else conclusion[1:]
            if negation in self.facts:
                return
            self.rules.append((premises, conclusion.strip()))
        else:
            negation = f"-{sentence}" if sentence[0] != "-" else sentence[1:]
            if negation in self.facts:
                print(f"[Warnung] Widerspruch erkannt: {sentence} widerspricht {negation}")
                return
            self.facts.add(sentence)

    def ask(self, query):
        negation = f"-{query}" if query[0] != "-" else query[1:]
        if negation in self.facts:
            return False

        inferred = set(self.facts)
        changed = True
        while changed:
            changed = False
            for premises, conclusion in self.rules:
                if premises <= inferred and conclusion not in inferred:
                    negation = f"-{conclusion}" if conclusion[0] != "-" else conclusion[1:]
                    if negation in inferred:
                        continue
                    inferred.add(conclusion)
                    changed = True

        if query.startswith("W"):
            possible_wumpus_positions = [f"W{i}{j}" for i in range(1, 5) for j in range(1, 5)]
            inferred_wumpus_positions = [pos for pos in possible_wumpus_positions if pos in inferred]
            if len(inferred_wumpus_positions) > 1:
                return False

        if query.startswith("P"):
            pos = query[1:]
            x, y = int(pos[0]) - 1, int(pos[1]) - 1
            neighbor_positions = get_neighbors(x, y)
            neighbor_pos = [f"{nx + 1}{ny + 1}" for nx, ny in neighbor_positions]
            must_be_pit = False
            for npos in neighbor_pos:
                if f"B{npos}" in inferred:
                    npos_x, npos_y = int(npos[0]) - 1, int(npos[1]) - 1
                    pit_candidates = [f"P{nx + 1}{ny + 1}" for nx, ny in get_neighbors(npos_x, npos_y) if f"-P{nx + 1}{ny + 1}" not in inferred]
                    if query not in pit_candidates:
                        return False 
                    if len(pit_candidates) == 1 and pit_candidates[0] == query:
                        must_be_pit = True 
                elif f"-B{npos}" in inferred and query in [f"P{nx + 1}{ny + 1}" for nx, ny in get_neighbors(int(npos[0]) - 1, int(npos[1]) - 1)]:
                    return False 
            return must_be_pit

        return query in inferred


In [4]:
ACTIONS = {"walk": 0, "turn_left": 1, "turn_right": 2, "shoot": 3, "grab": 4, "climb": 5}


In [5]:
def update_kb_horn(kb, obs, x, y):
    pos = f"{x + 1}{y + 1}"

    kb.tell(f"S{pos}" if obs["stench"] else f"-S{pos}")
    kb.tell(f"B{pos}" if obs["breeze"] else f"-B{pos}")
    kb.tell(f"G{pos}" if obs["glitter"] else f"-G{pos}")

    kb.tell(f"-W{pos}")  # kein Wumpus auf aktuellem Feld
    kb.tell(f"-P{pos}")  # keine Grube auf aktuellem Feld

    neighbors = get_neighbors(x, y)
    neighbor_pos = [f"{nx + 1}{ny + 1}" for nx, ny in neighbors]

    if obs["stench"]:
        for npos in neighbor_pos:
            if f"-W{npos}" not in kb.facts:
                kb.tell(f"S{pos} => W{npos}")
    else:
        for npos in neighbor_pos:
            kb.tell(f"-W{npos}")

    if obs["breeze"]:
        for npos in neighbor_pos:
            if f"-P{npos}" not in kb.facts:
                kb.tell(f"B{pos} => P{npos}")
    else:
        for npos in neighbor_pos:
            kb.tell(f"-P{npos}")

    for npos in neighbor_pos:
        if kb.ask(f"W{npos}"):
            print(f"[Sicher erkannt] Wumpus in Feld {npos}")
        if kb.ask(f"P{npos}"):
            print(f"[Sicher erkannt] Pit in Feld {npos}")

In [6]:
def move_and_update(env, kb, action, x, y, direction):
    obs, reward, done, _ = env.step(action)
    env.render()

    new_x, new_y, new_direction = x, y, direction

    if action == ACTIONS["walk"]:
        candidate_x, candidate_y = x, y
        if direction == 0:  # Osten
            candidate_x += 1
        elif direction == 1:  # Süden
            candidate_y -= 1
        elif direction == 2:  # Westen
            candidate_x -= 1
        elif direction == 3:  # Norden
            candidate_y += 1

        if 0 <= candidate_x < 4 and 0 <= candidate_y < 4:
            new_x, new_y = candidate_x, candidate_y
            print(f"Agent läuft von [{x+1},{y+1}] nach [{new_x+1},{new_y+1}]")
        else:
            print("[WARNUNG] Versuch, aus der Welt zu laufen – Koordinaten bleiben gleich.")

    elif action == ACTIONS["turn_right"]:
        new_direction = (direction + 1) % 4
    elif action == ACTIONS["turn_left"]:
        new_direction = (direction - 1) % 4

    richtung_namen = ["Ost", "Süd", "West", "Nord"]
    print(f"Ausrichtung nach: {richtung_namen[new_direction]}")

    if new_x != x or new_y != y:
        update_kb_horn(kb, obs, new_x, new_y)
        
    return obs, new_x, new_y, new_direction, done, reward


In [7]:
def reset_world_and_kb(env):
    obs = env.reset()
    env.render()
    kb = KnowledgeBase_Horn()
    return obs, kb, 0, 0, 0

In [8]:
def test_1_startfeld(env):
    print("\nTest 1: Nur vom Startfeld aus – Wumpus eingrenzen\n")
    obs, kb, x, y, direction = reset_world_and_kb(env)
    update_kb_horn(kb, obs, 0, 0)
    print("Geruch am Start?", "S11" in kb.facts or any("S11" in clause for clause in kb.cnf_clauses))
    print("Kann W21 gefolgert werden?", kb.ask("W21"))
    print("Kann W12 gefolgert werden?", kb.ask("W12"))
    print("Erwartung: Beides False, da unklar ob Rechts oder oben.")

In [9]:
def test_2_wumpus_bei_1_2(env):
    print("\nTest 2: Nach oben laufen (Feld [1,2]) und Wumpus suchen\n")
    obs, kb, x, y, direction = reset_world_and_kb(env)
    update_kb_horn(kb, obs, x, y)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["turn_left"], x, y, direction)
    print("Kann W21 gefolgert werden?", kb.ask("W21"))
    print("Kann W12 gefolgert werden?", kb.ask("W12"))
    print("Erwartung: Beides False, da unklar ob Rechts oder oben.")
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    print("Kann W21 gefolgert werden?", kb.ask("W21"))
    print("Kann W12 gefolgert werden?", kb.ask("W12"))
    print("Erwartung: W21=True, W12=False")

In [10]:
def test_3_pit_bei_3_1(env):
    print("\nTest 3: Gehe zu [3,1] und schaue ob das Pit erkannt wird\n")
    obs, kb, x, y, direction = reset_world_and_kb(env)
    update_kb_horn(kb, obs, x, y)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["turn_left"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)

    print("Pit in [1,3]?", kb.ask("P13"))
    print("Pit in [2,2]?", kb.ask("P22"))

    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    print("Pit in [1,4]?", kb.ask("P14"))
    print("Pit in [2,3]?", kb.ask("P23"))
    print("Erwartung: p14=False, p23=False da man sich nicht sicher sein kann")

In [11]:
def test_4_pit_herumgehen(env):
    print("\nTest 4: Gehe zu [3,1] und laufe um das Pit\n")
    obs, kb, x, y, direction = reset_world_and_kb(env)
    update_kb_horn(kb, obs, x, y)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["turn_left"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)

    print("Pit in [1,3]?", kb.ask("P13"))
    print("Pit in [2,2]?", kb.ask("P22"))

    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    print("Pit in [1,4]?", kb.ask("P14"))
    print("Pit in [2,3]?", kb.ask("P23"))

    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    print("Pit in [1,4]?", kb.ask("P14"))
    print("Pit in [2,3]?", kb.ask("P23"))

    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["turn_right"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)

    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["turn_right"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)

    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["turn_right"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["turn_right"], x, y, direction)
    obs, x, y, direction, _, _ = move_and_update(env, kb, ACTIONS["walk"], x, y, direction)


    print("Pit in [1,4]?", kb.ask("P14"))
    print("Pit in [2,3]?", kb.ask("P23"))
    print("Erwartung: p14=False, p23=true da man sich sicher sein kann")

In [12]:
env = gym.make("Wumpus-v0", disable_env_checker=True)
obs = env.reset()
env.render()
kb = KnowledgeBase_Horn()
test_1_startfeld(env)
env.close()


+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0

Test 1: Nur vom Startfeld aus – Wumpus eingrenzen

+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0
Geruch am Start? True
Kann W21 gefolgert werden? False
Kann W12 gefolgert werden? False
Erwartung: Beides False, da unklar ob Rechts oder oben.
Not necessary since no seperate window was opened


In [13]:
env = gym.make("Wumpus-v0", disable_env_checker=True)
obs = env.reset()
env.render()
kb = KnowledgeBase_Horn()
test_2_wumpus_bei_1_2(env)
env.close()


+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0

Test 2: Nach oben laufen (Feld [1,2]) und Wumpus suchen

+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0
+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A^|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : -1
Ausricht

In [14]:
env = gym.make("Wumpus-v0", disable_env_checker=True)
obs = env.reset()
env.render()
kb = KnowledgeBase_Horn()
test_3_pit_bei_3_1(env)
env.close()


+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0

Test 3: Gehe zu [3,1] und schaue ob das Pit erkannt wird

+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0
+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A^|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : -1
Ausrich

In [15]:
env = gym.make("Wumpus-v0", disable_env_checker=True)
obs = env.reset()
env.render()
kb = KnowledgeBase_Horn()
test_4_pit_herumgehen(env)
env.close()


+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0

Test 4: Gehe zu [3,1] und laufe um das Pit

+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A>|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : 0
+---+---+---+---+
|   |   |   |  G|
|   |   |   |   |
+---+---+---+---+
|   | P |   |   |
|   |   |   |   |
+---+---+---+---+
|   |   |   |   |
|   |   |   |   |
+---+---+---+---+
|   |W  |   |   |
| A^|   |   |   |
+---+---+---+---+
Perception [ St: True, Br: False, G: False, Bu: False, Sc: False ]
Score : -1
Ausrichtung nach: Nor